# PRAKTIKUM 1
**Bagging dengan RandomForest**

Pada kasus ini kita akan menggunakan salah satu metode bagging yaitu RandomForest untuk mengklasifikasikan jenis tumor. Dalam latihan ini Anda akan melakukan training dengan data Wisconsin Breast Cancer Dataset dari UCI machine learning repository. Latihan ini akan melakukan prediksi memprediksi apakah tumor ganas atau jinak.

Kita akan membandingkan performa dari algoritma Decision Tree dan RandomForest pada kasus ini.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Import Library**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # import DT
from sklearn.ensemble import RandomForestClassifier # import RandomForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

- import numpy as np: Mengimpor library NumPy dan memberikan alias np. NumPy digunakan untuk operasi numerik di Python, seperti array multidimensi.
- import pandas as pd: Mengimpor library Pandas dan memberikan alias pd. Pandas digunakan untuk manipulasi dan analisis data, khususnya untuk struktur data DataFrame.
- from sklearn.tree import DecisionTreeClassifier: Mengimpor kelas DecisionTreeClassifier dari modul sklearn.tree. Kelas ini digunakan untuk membuat model Decision Tree. Decision Tree adalah algoritma machine learning yang digunakan untuk klasifikasi dan regresi.
- from sklearn.ensemble import RandomForestClassifier: Mengimpor kelas RandomForestClassifier dari modul sklearn.ensemble. Kelas ini digunakan untuk membuat model Random Forest. Random Forest adalah algoritma machine learning yang merupakan ensemble dari Decision Tree, yang meningkatkan akurasi dan mengurangi overfitting.
- from sklearn.model_selection import train_test_split: Mengimpor fungsi train_test_split dari modul sklearn.model_selection. Fungsi ini digunakan untuk membagi dataset menjadi data training dan data testing.
- from sklearn.metrics import accuracy_score, classification_report: Mengimpor fungsi accuracy_score dan classification_report dari modul sklearn.metrics. Fungsi-fungsi ini digunakan untuk mengevaluasi performa model klasifikasi. accuracy_score menghitung akurasi model, dan classification_report memberikan laporan yang lebih detail tentang performa model, seperti precision, recall, dan F1-score.

**Persiapan Data**

In [ ]:
# Load data
df = pd.read_csv('/content/drive/MyDrive/machine learning/js5/wbc.csv')

df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


- df = pd.read_csv() : digunakan untuk memuat data dari file csv ke dalam Pandas DataFrame df
- df.head() : menampilkan 5 baris pertama DataFrame

In [ ]:
# Cek kolom null
df.isnull().sum()

,0
id,0
diagnosis,0
radius_mean,0
texture_mean,0
perimeter_mean,0
area_mean,0
smoothness_mean,0
compactness_mean,0
concavity_mean,0
concave points_mean,0


Menghitung jumlah nilai yang hilang (missing values) di setiap kolom DataFrame df.

- df.isnull(): Mengembalikan DataFrame boolean dengan nilai True untuk setiap sel yang mengandung nilai yang hilang (NaN) dan False untuk sel yang tidak hilang.
- .sum(): Menjumlahkan nilai True di setiap kolom (True dianggap sebagai 1 dan False sebagai 0). Hasilnya adalah jumlah nilai yang hilang di setiap kolom.

In [ ]:
# Seleksi fitur

# Slice dataframe mulai dari kolom 'radius_mean' sampai 'fractal_dimension_worst'
X = df.iloc[:,3:-1]
y = df['diagnosis']
y = y.map({'M':1, 'B':0}) # Encode label

# Cek jumlah fitur dan instance
X.shape

(569, 29)

- X = df.iloc[:,3:-1]: Membuat variabel X yang berisi semua baris dan kolom dari kolom ke-4 hingga kolom kedua dari belakang DataFrame df. Variabel ini akan digunakan sebagai fitur (input) untuk model.
- y = df['diagnosis']: Membuat variabel y yang berisi kolom diagnosis dari DataFrame df. Variabel ini akan digunakan sebagai target (output) untuk model.
- y = y.map({'M':1, 'B':0}): Melakukan encoding pada kolom diagnosis dengan mengubah nilai 'M' menjadi 1 dan 'B' menjadi 0. Encoding ini diperlukan karena sebagian besar algoritma machine learning hanya menerima input numerik.
X.shape: Menampilkan dimensi dari variabel X, yaitu jumlah instance (baris) dan jumlah fitur (kolom).
- Output dari kode diatas adalah  memisahkan fitur dan target, serta melakukan encoding pada target agar siap digunakan untuk training model.

**Split data training dan testing**

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

Kode ini membagi data menjadi set training dan set testing menggunakan fungsi train_test_split dari scikit-learn (sklearn).

- X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1): Membagi data fitur (X) dan data target (y) menjadi empat bagian:

  - X_train: Data fitur untuk training.
  - X_test: Data fitur untuk testing.
  - y_train: Data target untuk training.
  - y_test: Data target untuk testing.

- test_size=0.2: Menentukan bahwa 20% dari data akan digunakan sebagai data testing dan 80% sebagai data training.

- random_state=1: Menjamin bahwa pembagian data akan sama setiap kali kode dijalankan, sehingga hasilnya dapat direproduksi.

Pembagian data ini penting untuk mengevaluasi performa model machine learning pada data yang belum pernah dilihat sebelumnya.

**Traning Decision Tree**

In [ ]:
# Secara default, DecisionTreeClassifier dari scikit-learn akan menggunakan nilai "Gini" untuk kriteria
# Terdapat beberapa "hyperparamater" yang dapat digunakan. Silahka baca dokumentasi
# Pada kasus ini kita akan menggunakan parameter default
dt = DecisionTreeClassifier()

# Sesuaikan dt ke set training
dt.fit(X_train, y_train)

# Memprediksi label set test
y_pred_dt = dt.predict(X_test)

#  menghitung set accuracy
acc_dt = accuracy_score(y_test, y_pred_dt)
print("Test set accuracy: {:.2f}".format(acc_dt))
print(f"Test set accuracy: {acc_dt}")

Test set accuracy: 0.96
Test set accuracy: 0.956140350877193


- dt = DecisionTreeClassifier(): Membuat objek dt dari kelas DecisionTreeClassifier. Ini adalah model Decision Tree yang akan digunakan.
- dt.fit(X_train, y_train): Melatih model Decision Tree (dt) menggunakan data training (X_train dan y_train). Proses ini akan membuat struktur pohon keputusan berdasarkan pola dalam data training.
- y_pred_dt = dt.predict(X_test): Melakukan prediksi pada data testing (X_test) menggunakan model yang telah dilatih. Hasil prediksi disimpan dalam variabel y_pred_dt.
- acc_dt = accuracy_score(y_test, y_pred_dt): Menghitung akurasi model dengan membandingkan prediksi (y_pred_dt) dengan nilai target sebenarnya (y_test).
- print("Test set accuracy: {:.2f}".format(acc_dt)): Menampilkan akurasi model dalam format dua angka di belakang koma.
- print(f"Test set accuracy: {acc_dt}"): Menampilkan akurasi model.

**Training RandomForest**

In [ ]:
# Pada kasus kali ini kita akan menggunakan estimator pada RandomForest
# Untuk detail parameter (hyperparameter) silahkan cek dokumentasi

rf = RandomForestClassifier(n_estimators=10, random_state=1)

# Sesuaikan dt ke set training
rf.fit(X_train, y_train)

# Memprediksi label set test
y_pred_rf = rf.predict(X_test)

#  menghitung set accuracy
acc_rf = accuracy_score(y_test, y_pred_rf)
print("Test set accuracy: {:.2f}".format(acc_rf))
print(f"Test set accuracy: {acc_rf}")

Test set accuracy: 0.96
Test set accuracy: 0.956140350877193


- rf = RandomForestClassifier(n_estimators=10, random_state=1): Membuat objek rf dari kelas RandomForestClassifier. Ini adalah model Random Forest yang akan digunakan.

  - n_estimators=10: Menentukan jumlah pohon keputusan (estimators) dalam Random Forest, yaitu 10.
  - random_state=1: Menjamin bahwa hasil training akan sama setiap kali kode dijalankan.

- rf.fit(X_train, y_train): Melatih model Random Forest (rf) menggunakan data training.

- y_pred_rf = rf.predict(X_test): Melakukan prediksi pada data testing menggunakan model Random Forest yang telah dilatih.

- acc_rf = accuracy_score(y_test, y_pred_rf): Menghitung akurasi model Random Forest.

- print("Test set accuracy: {:.2f}".format(acc_rf)): Menampilkan akurasi model dalam format dua angka di belakang koma.

- print(f"Test set accuracy: {acc_rf}"): Menampilkan akurasi model.

# PRAKTIKUM 2
**Boosting dengan AdaBoost**

Pada kasus ini kita akan menggunakan salah satu metode boosting yaitu AdaBoost untuk mengklasifikasikan jenis bunga Iris. Dalam latihan ini kita akan menggunakan dataset Iris yang sangat lazim digunakan. Latihan ini akan melakukan prediksi memprediksi 3 jenis bunga Iris yaitu, Iris Setosa, Iris Versicolor, dan Iris Virginica berdasarkan panjang dan lebar sepal dan petal.

Kita akan membandingkan performa dari algoritma Decision Tree dan AdaBoost pada kasus ini.

**Import Library**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # import DT
from sklearn.ensemble import AdaBoostClassifier # import AdaBoost
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder # Kebutuhan encoding label

- import numpy as np: Mengimpor library NumPy dan memberikan alias np. NumPy digunakan untuk operasi numerik, khususnya untuk array multidimensi.
- import pandas as pd: Mengimpor library Pandas dan memberikan alias pd. Pandas digunakan untuk manipulasi dan analisis data, terutama struktur data DataFrame.
- from sklearn.tree import DecisionTreeClassifier: Mengimpor kelas DecisionTreeClassifier dari modul sklearn.tree. Kelas ini digunakan untuk membuat model Decision Tree, yang sering dipakai sebagai base estimator untuk AdaBoost.
- from sklearn.ensemble import AdaBoostClassifier: Mengimpor kelas AdaBoostClassifier dari modul sklearn.ensemble. Kelas ini digunakan untuk membuat model AdaBoost. AdaBoost meningkatkan performa model dengan menggabungkan beberapa weak learner, seperti Decision Tree.
- from sklearn.model_selection import train_test_split: Mengimpor fungsi train_test_split dari modul sklearn.model_selection. Fungsi ini digunakan untuk membagi dataset menjadi data training dan testing, yang penting untuk evaluasi model.
- from sklearn.metrics import accuracy_score, classification_report: Mengimpor fungsi accuracy_score dan classification_report dari modul sklearn.metrics. Fungsi-fungsi ini digunakan untuk mengevaluasi performa model.
- from sklearn.preprocessing import LabelEncoder: Mengimpor kelas LabelEncoder dari modul sklearn.preprocessing. Kelas ini digunakan untuk encoding label, mengubah nilai kategorikal menjadi numerik agar bisa diproses algoritma machine learning.


**Persiapan Data**

In [ ]:
# Load data
df = pd.read_csv('/content/drive/MyDrive/machine learning/js5/iris.csv')

df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa



- df = pd.read_csv() : digunakan untuk memuat data dari file csv ke dalam Pandas DataFrame df
- df.head() : menampilkan 5 baris pertama DataFrame

In [ ]:
# Cek kolom null
df.isnull().sum()

,0
Id,0
SepalLengthCm,0
SepalWidthCm,0
PetalLengthCm,0
PetalWidthCm,0
Species,0


Menghitung jumlah nilai yang hilang (missing values) di setiap kolom DataFrame df.

- df.isnull(): Mengembalikan DataFrame boolean dengan nilai True untuk setiap sel yang mengandung nilai yang hilang (NaN) dan False untuk sel yang tidak hilang.
- .sum(): Menjumlahkan nilai True di setiap kolom (True dianggap sebagai 1 dan False sebagai 0). Hasilnya adalah jumlah nilai yang hilang di setiap kolom.

In [ ]:
# Seleksi fitur
X = df.iloc[:,2:-1]
y = df['Species']

# encode label
ec = LabelEncoder()
y = ec.fit_transform(y)

# Cek jumlah fitur dan instance
print(X.shape)

# Cek label
print(y)

(150, 3)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


- X = df.iloc[:,2:-1]: Membuat variabel X yang berisi fitur (input) untuk model. iloc digunakan untuk memilih data berdasarkan indeks. Kode ini memilih semua baris dan kolom dari kolom ketiga hingga kolom kedua dari belakang.
- y = df['Species']: Membuat variabel y yang berisi kolom 'Species', yang akan menjadi target (output) model.
- ec = LabelEncoder(): Membuat objek ec dari kelas LabelEncoder(). Objek ini akan digunakan untuk mengubah label kategorikal menjadi numerik.
- y = ec.fit_transform(y): Melakukan encoding pada variabel y menggunakan ec. fit_transform akan mempelajari mapping dari label kategorikal ke numerik dan langsung menerapkannya pada y.
- print(X.shape): Menampilkan dimensi dari X, yaitu jumlah baris (instance) dan jumlah kolom (fitur).
- print(y): Menampilkan nilai y setelah encoding. Sekarang labelnya berupa angka.

**Split data training dan testing**

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

Kode ini membagi data (fitur X dan target y) menjadi set training dan set testing menggunakan train_test_split.

- X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1): Membagi data menjadi empat bagian:
  - data fitur untuk training (X_train)
  - data fitur untuk testing (X_test)
  - data target untuk training (y_train)
  - data target untuk testing (y_test)

- test_size=0.2: Menentukan proporsi data yang akan digunakan untuk testing, yaitu 20%. Sisanya (80%) akan digunakan untuk training.

- random_state=1: Menjamin pembagian data yang konsisten setiap kali kode dijalankan. Ini penting untuk reproducibility hasil eksperimen.

**Training Decision Tree**

In [ ]:
# Secara default, DecisionTreeClassifier dari scikit-learn akan menggunakan nilai "Gini" untuk kriteria
# Terdapat beberapa "hyperparamater" yang dapat digunakan. Silahka baca dokumentasi
# Pada kasus ini kita akan menggunakan parameter default
dt = DecisionTreeClassifier()

# Sesuaikan dt ke set training
dt.fit(X_train, y_train)

# Memprediksi label set test
y_pred_dt = dt.predict(X_test)

#  menghitung set accuracy
acc_dt = accuracy_score(y_test, y_pred_dt)
print("Test set accuracy: {:.2f}".format(acc_dt))
print(f"Test set accuracy: {acc_dt}")

Test set accuracy: 0.97
Test set accuracy: 0.9666666666666667


Kode ini melatih model Decision Tree Classifier dan mengevaluasi kinerjanya:

- dt = DecisionTreeClassifier(): Membuat objek dt dari kelas DecisionTreeClassifier. Ini menginisialisasi model Decision Tree dengan parameter default.

- dt.fit(X_train, y_train): Melatih model dt menggunakan data training (X_train dan y_train). Proses ini membangun struktur pohon keputusan berdasarkan pola-pola dalam data training.

- y_pred_dt = dt.predict(X_test): Menggunakan model dt yang sudah dilatih untuk memprediksi label pada data testing (X_test). Hasil prediksi disimpan di y_pred_dt.

- acc_dt = accuracy_score(y_test, y_pred_dt): Menghitung akurasi model dengan membandingkan prediksi (y_pred_dt) dengan nilai sebenarnya (y_test).

- print("Test set accuracy: {:.2f}".format(acc_dt)): Menampilkan akurasi model, diformat dengan dua angka di belakang koma.

- print(f"Test set accuracy: {acc_dt}"): Menampilkan akurasi model.

**Training AdaBoost**

In [ ]:
# Pada kasus kali ini kita akan menggunakan estimator pada AdaBoost
# Untuk detail parameter (hyperparameter) silahkan cek dokumentasi

ada = AdaBoostClassifier(n_estimators=2)

# Sesuaikan dt ke set training
ada.fit(X_train, y_train)

# Memprediksi label set test
y_pred_ada = ada.predict(X_test)

#  menghitung set accuracy
acc_ada = accuracy_score(y_test, y_pred_ada)
print("Test set accuracy: {:.2f}".format(acc_ada))
print(f"Test set accuracy: {acc_ada}")

Test set accuracy: 0.97
Test set accuracy: 0.9666666666666667


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Kode ini melatih model AdaBoost Classifier dan mengevaluasi performanya. Berikut penjelasannya:

- ada = AdaBoostClassifier(n_estimators=2): Membuat objek ada dari kelas AdaBoostClassifier.
  - n_estimators=2: Menentukan jumlah estimator (dalam hal ini, kemungkinan besar Decision Tree) yang digunakan dalam model AdaBoost.
- ada.fit(X_train, y_train): Melatih model ada menggunakan data training (X_train dan y_train). Proses training AdaBoost melibatkan iterasi untuk meningkatkan bobot instance yang salah diklasifikasi oleh estimator sebelumnya.
- y_pred_ada = ada.predict(X_test): Menggunakan model ada yang sudah dilatih untuk memprediksi label pada data testing (X_test).
- acc_ada = accuracy_score(y_test, y_pred_ada): Menghitung akurasi model dengan membandingkan prediksi (y_pred_ada) dengan nilai sebenarnya (y_test).
- print("Test set accuracy: {:.2f}".format(acc_ada)): Menampilkan akurasi model, diformat dengan dua angka di belakang koma.
- print(f"Test set accuracy: {acc_ada}"): Menampilkan akurasi model.

# PRAKTIKUM 3

**Stacking**

Lengkapi bagian berikut dengan data sesuai tugas, dan tentukan perbedaan nilai akurasi antara Random Forest, Adaboost, dan Stacking

In [ ]:
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier



layer_one_estimators = [
                        ('rf_1', RandomForestClassifier(n_estimators=10, random_state=42)),
                        ('knn_1', KNeighborsClassifier(n_neighbors=5))
                       ]
layer_two_estimators = [
                        ('dt_2', DecisionTreeClassifier()),
                        ('rf_2', RandomForestClassifier(n_estimators=50, random_state=42)),
                       ]
layer_two = StackingClassifier(estimators=layer_two_estimators, final_estimator=LogisticRegression())


clf = StackingClassifier(estimators=layer_one_estimators, final_estimator=layer_two)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)
clf.fit(X_train, y_train).score(X_test, y_test)

0.8947368421052632

- from sklearn.ensemble import RandomForestClassifier, StackingClassifier: Mengimpor RandomForestClassifier dan StackingClassifier dari modul sklearn.ensemble.

  - RandomForestClassifier adalah kelas untuk membuat model Random Forest, sebuah algoritma ensemble learning yang menggabungkan multiple decision tree.
  - StackingClassifier adalah kelas untuk membuat model Stacking, teknik ensemble learning yang menggabungkan beberapa model machine learning dengan cara bertingkat.

- from sklearn.linear_model import LogisticRegression: Mengimpor LogisticRegression dari modul sklearn.linear_model. LogisticRegression adalah kelas untuk membuat model Logistic Regression, algoritma yang digunakan untuk klasifikasi.

- from sklearn.model_selection import train_test_split: Mengimpor train_test_split dari modul sklearn.model_selection. Fungsi ini digunakan untuk membagi dataset menjadi data training dan data testing.

- from sklearn.neighbors import KNeighborsClassifier: Mengimpor KNeighborsClassifier dari modul sklearn.neighbors. KNeighborsClassifier adalah kelas untuk membuat model K-Nearest Neighbors (KNN), algoritma yang digunakan untuk klasifikasi dan regresi.

- from sklearn.tree import DecisionTreeClassifier: Mengimpor DecisionTreeClassifier dari modul sklearn.tree. DecisionTreeClassifier adalah kelas untuk membuat model Decision Tree, algoritma yang digunakan untuk klasifikasi dan regresi.

1. layer_one_estimators = [
    ('rf_1', RandomForestClassifier(n_estimators=10, random_state=42)),
    ('knn_1', KNeighborsClassifier(n_neighbors=5))             
]
Layer pertama terdiri dari dua model:
  - RandomForestClassifier dengan 10 estimators (pohon keputusan).
  - KNeighborsClassifier yang menggunakan 5 tetangga terdekat untuk klasifikasi (k-nearest neighbors).

  Keduanya diberi nama (alias) 'rf_1' dan 'knn_1', sehingga mudah dikenali.
2.
layer_two_estimators = [
    ('dt_2', DecisionTreeClassifier()),
    ('rf_2', RandomForestClassifier(n_estimators=50, random_state=42)),
]
Layer kedua juga terdiri dari dua model:
  - DecisionTreeClassifier, yang merupakan model pohon keputusan.
  - RandomForestClassifier dengan 50 pohon keputusan.

  Ini adalah model yang akan dipasangkan dalam layer dua sebagai prediktor tambahan.
3.
layer_two = StackingClassifier(estimators=layer_two_estimators, final_estimator=LogisticRegression())
  - StackingClassifier digunakan untuk menggabungkan hasil prediksi dari model-model di layer two.
  - LogisticRegression berperan sebagai model final (atau meta-model), yang memproses hasil dari layer kedua untuk membuat prediksi akhir. Artinya, output dari DecisionTreeClassifier dan RandomForestClassifier pada layer kedua digunakan sebagai input untuk Logistic Regression.
4.
clf = StackingClassifier(estimators=layer_one_estimators, final_estimator=layer_two)
  - Di sini, layer_one_estimators digabungkan menggunakan StackingClassifier.
  - layer_two yang sudah ditentukan sebelumnya bertindak sebagai meta-model untuk layer pertama. Jadi, output dari Random Forest dan KNeighbors dari layer pertama digunakan sebagai input untuk layer_two, yang kemudian memproses hasilnya menggunakan kombinasi Decision Tree dan Random Forest.
5.
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)
clf.fit(X_train, y_train).score(X_test, y_test)
  - train_test_split membagi dataset X (fitur) dan y (label) menjadi data latih (X_train, y_train) dan data uji (X_test, y_test). Metode stratify=y memastikan bahwa distribusi kelas yang ada di data latih dan data uji tetap sama (berguna untuk dataset yang tidak seimbang).
  - clf.fit(X_train, y_train) melatih model stacking pada data latih.
  - .score(X_test, y_test) menghitung skor akurasi model pada data uji, yaitu perbandingan antara prediksi model dan label sebenarnya di data uji.

perbedaan nilai akurasi antara Random Forest, Adaboost, dan Stacking

# PRAKTIKUM 4

**Stacking dengan Voting**

Pada kasus ini kita akan menggunakan salah satu metode stacking yaitu voting untuk mengklasifikasikan pasien penderita diabetes dengan beberapa ciri. Pasien akan di klasifikasikan menjadi pasien menderita diabetes (1) dan tidak menderita diabetes (0). Pertama-tama, kita akan menggunakan beberapa algoritma klasifikasi secara terpisah, yaitu Naive Bayes, SVM Linier, dan SVM RBF. Setelah itu, kita akan menggabungkan performa dari 3 algoritma tersebut dengan menggunakan metode ensemble voting.

**Import Library**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB # import Naive Bayes model Gaussian (asumsi data terdistribusi normal)
from sklearn.svm import SVC # import SVM classifier
from sklearn.ensemble import VotingClassifier # import model Voting
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

- import numpy as np: Mengimpor library NumPy dan memberikan alias np, digunakan untuk operasi numerik seperti array multidimensi.
- import pandas as pd: Mengimpor library Pandas dan memberikan alias pd, digunakan untuk manipulasi dan analisis data, khususnya struktur data DataFrame.
- from sklearn.naive_bayes import GaussianNB: Mengimpor kelas GaussianNB dari modul sklearn.naive_bayes, digunakan untuk membuat model Naive Bayes dengan asumsi data berdistribusi normal.
- from sklearn.svm import SVC: Mengimpor kelas SVC dari modul sklearn.svm, digunakan untuk membuat model Support Vector Machine (SVM) classifier.
- from sklearn.ensemble import VotingClassifier: Mengimpor kelas VotingClassifier dari modul sklearn.ensemble, digunakan untuk membuat model ensemble Voting. Voting Classifier menggabungkan prediksi dari beberapa model untuk meningkatkan performa.
- from sklearn.model_selection import train_test_split: Mengimpor fungsi train_test_split dari modul sklearn.model_selection, digunakan untuk membagi dataset menjadi data training dan data testing.
- from sklearn.metrics import accuracy_score, classification_report: Mengimpor fungsi accuracy_score dan classification_report dari modul sklearn.metrics, digunakan untuk mengevaluasi performa model.

**Persiapan Data**

In [ ]:
# Load Data

dbt = pd.read_csv('/content/drive/MyDrive/machine learning/js5/diabetes.csv')

dbt.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


- dbt = pd.read_csv() : digunakan untuk memuat data dari file csv ke dalam Pandas DataFrame dbt
- dbt.head() : menampilkan 5 baris pertama DataFrame

In [ ]:
# Cek nama kolom
dbt.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

- dbt.columns akan menampilkan nama-nama kolom yang ada pada DataFrame dbt.

In [ ]:
# Cek kolom null
dbt.isnull().sum()

,0
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0
Outcome,0


- dbt.isnull(): Mengembalikan DataFrame boolean dengan nilai True untuk setiap sel yang mengandung nilai yang hilang (NaN) dan False untuk sel yang tidak hilang.
- .sum(): Menjumlahkan nilai True di setiap kolom (True dianggap sebagai 1 dan False sebagai 0). Hasilnya adalah jumlah nilai yang hilang di setiap kolom.

In [ ]:
# Pada kasus ini, agak tidak masuk akal jika beberapa parameter bernilai 0
# sebagai contoh adalah nilai 'Glucose', 'BloodPlessure' ataupun 'Insulin'.
# Sekecil apapun nilainya, setiap manusia yang hidup pasti miliki nilai-nilai tersebut

# Kita akan manipulasi nilai yang 0 dengan melakukan 'imputasi' atau mengganti nilainya dengan nilai sintetis
# Pada kasus ini, kita akan menggunakan nilai mean

# Cek kolom neng nilai 0
feature_columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
for column in feature_columns:
    print("============================================")
    print(f"{column} ==> Missing zeros : {len(dbt.loc[dbt[column] == 0])}")

Pregnancies ==> Missing zeros : 111
Glucose ==> Missing zeros : 5
BloodPressure ==> Missing zeros : 35
SkinThickness ==> Missing zeros : 227
Insulin ==> Missing zeros : 374
BMI ==> Missing zeros : 11
DiabetesPedigreeFunction ==> Missing zeros : 0
Age ==> Missing zeros : 0


- feature_columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']: Mendefinisikan list feature_columns yang berisi nama-nama kolom fitur yang ingin diperiksa.
- for column in feature_columns:: Looping untuk setiap kolom dalam feature_columns.
  - print("============================================"): Menampilkan garis pemisah untuk memperjelas output.
  - print(f"{column} ==> Missing zeros : {len(dbt.loc[dbt[column] == 0])}"): Menampilkan nama kolom dan jumlah nilai nol yang hilang pada kolom tersebut.
    - dbt.loc[dbt[column] == 0]: Memilih baris di dbt di mana nilai kolom column sama dengan 0.
    - len(...): Menghitung jumlah baris yang terpilih.

Kode ini membantu dalam identifikasi data yang hilang yang direpresentasikan sebagai nilai nol. Nilai nol pada beberapa kolom, seperti 'Glucose' atau 'BloodPressure', mungkin mengindikasikan data yang hilang dan perlu ditangani sebelum digunakan untuk training model machine learning.

In [ ]:
# Input nilai 0 dengan mean
from sklearn.impute import SimpleImputer

fill_values = SimpleImputer(missing_values=0, strategy="mean", copy=False)

dbt[feature_columns] = fill_values.fit_transform(dbt[feature_columns])

Kode ini menangani nilai yang hilang (missing values) yang direpresentasikan sebagai 0 pada kolom-kolom fitur dalam DataFrame dbt menggunakan SimpleImputer dari scikit-learn.

- from sklearn.impute import SimpleImputer: Mengimpor kelas SimpleImputer dari modul sklearn.impute. SimpleImputer digunakan untuk mengisi nilai yang hilang dengan strategi sederhana seperti mean, median, atau modus.

- fill_values = SimpleImputer(missing_values=0, strategy="mean", copy=False): Membuat objek fill_values dari kelas SimpleImputer dengan parameter:

  - missing_values=0: Menentukan bahwa nilai 0 dianggap sebagai nilai yang hilang.
  - strategy="mean": Menentukan bahwa nilai yang hilang akan diisi dengan nilai rata-rata (mean) dari kolom tersebut.
  - copy=False: Menentukan bahwa imputasi akan dilakukan secara inplace pada DataFrame asli.

- dbt[feature_columns] = fill_values.fit_transform(dbt[feature_columns]): Mengisi nilai yang hilang pada kolom-kolom yang ditentukan di feature_columns menggunakan objek fill_values.

- fit_transform: Metode ini akan menghitung nilai rata-rata dari setiap kolom (fit) dan kemudian mengganti nilai yang hilang dengan nilai rata-rata tersebut (transform).

**Split data training dan testing**

In [ ]:
X = dbt[feature_columns]
y = dbt.Outcome

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

memisahkan fitur dan target dari DataFrame dbt, kemudian membagi data menjadi set training dan testing:

- X = dbt[feature_columns]: Membuat variabel X yang berisi fitur-fitur (input) untuk model machine learning. Fitur-fitur ini diambil dari kolom-kolom yang ada di feature_columns.
- y = dbt.Outcome: Membuat variabel y yang berisi target (output) untuk model. Target diambil dari kolom 'Outcome'.
- X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42): Membagi data (X dan y) menjadi set training dan testing menggunakan fungsi train_test_split.
  - test_size=0.3: Menentukan bahwa 30% data akan digunakan untuk testing.
  - random_state=42: Menjamin pembagian data yang sama setiap kali kode dijalankan, sehingga hasilnya dapat direproduksi.

**Training dengan GaussianNB**

**Standarisasi Fitur**

In [ ]:
# Karena asumsi Gaussian NB adalah data terdistribusi secara normal,
# maka kita perlu melakukan standarisasi

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

# Standarisasi pada fitur di X_train dan X_test
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

- from sklearn.preprocessing import StandardScaler: Mengimpor kelas StandardScaler dari modul sklearn.preprocessing. StandardScaler digunakan untuk standarisasi data dengan mengubah setiap fitur agar memiliki rata-rata (mean) 0 dan standar deviasi 1.
- sc = StandardScaler(): Membuat objek sc dari kelas StandardScaler.
- X_train_std = sc.fit_transform(X_train): Melakukan standarisasi pada data training (X_train).
  - fit_transform: Menghitung mean dan standar deviasi dari X_train (fit) dan kemudian melakukan standarisasi data menggunakan nilai-nilai tersebut (transform).
- X_test_std = sc.transform(X_test): Melakukan standarisasi pada data testing (X_test) menggunakan mean dan standar deviasi yang telah dihitung dari data training.
  - transform: Melakukan standarisasi data menggunakan mean dan standar deviasi yang sudah dihitung sebelumnya.

**Training dan Evaluasi**

In [ ]:
# Buat obyek GaussianNB
gnb_std = GaussianNB()

# Fit dengan data yang telah di standarisasi
gnb_std.fit(X_train_std, y_train)

# Prediksi dengan data test
y_pred_gnb = gnb_std.predict(X_test_std)

# Evaluasi akurasi testing data
acc_gnb = accuracy_score(y_test, y_pred_gnb)

# Print hasil evaluasi
print("Test set accuracy: {:.2f}".format(acc_gnb))
print(f"Test set accuracy: {acc_gnb}")

Test set accuracy: 0.74
Test set accuracy: 0.7359307359307359


Kode ini melatih model Gaussian Naive Bayes (gnb_std) menggunakan data yang telah distandarisasi, kemudian mengevaluasi akurasi model pada data testing.

- gnb_std = GaussianNB(): Membuat objek gnb_std dari kelas GaussianNB. Ini menginisialisasi model Gaussian Naive Bayes dengan parameter default.
- gnb_std.fit(X_train_std, y_train): Melatih model gnb_std menggunakan data training yang telah distandarisasi (X_train_std dan y_train).
- y_pred_gnb = gnb_std.predict(X_test_std): Melakukan prediksi pada data testing yang telah distandarisasi (X_test_std) menggunakan model gnb_std yang telah dilatih. Hasil prediksi disimpan di y_pred_gnb.
- acc_gnb = accuracy_score(y_test, y_pred_gnb): Menghitung akurasi model dengan membandingkan prediksi (y_pred_gnb) dengan nilai sebenarnya (y_test).
- print("Test set accuracy: {:.2f}".format(acc_gnb)): Menampilkan akurasi model, diformat dengan dua angka di belakang koma.
- print(f"Test set accuracy: {acc_gnb}"): Menampilkan akurasi model.

**Training dengan SVM Linier**

In [ ]:
# Model SVM linier tanpa tunnning hyperparameter
svm_lin = SVC(kernel='linear')

# Fit ke model
svm_lin.fit(X_train_std, y_train)

# Prediksi
y_pred_svm_lin = svm_lin.predict(X_test_std)

# Evaluasi akurasi testing data
acc_svm_lin = accuracy_score(y_test, y_pred_svm_lin)

# Print hasil evaluasi
print("Test set accuracy: {:.2f}".format(acc_svm_lin))
print(f"Test set accuracy: {acc_svm_lin}")

Test set accuracy: 0.74
Test set accuracy: 0.7402597402597403


Kode ini melatih model Support Vector Machine (SVM) dengan kernel linear (svm_lin) menggunakan data yang telah distandarisasi, kemudian mengevaluasi akurasi model pada data testing.

- svm_lin = SVC(kernel='linear'): Membuat objek svm_lin dari kelas SVC dengan kernel linear.
  - kernel='linear': Menentukan bahwa model SVM akan menggunakan kernel linear untuk memisahkan data.
- svm_lin.fit(X_train_std, y_train): Melatih model svm_lin menggunakan data training yang telah distandarisasi.
- y_pred_svm_lin = svm_lin.predict(X_test_std): Melakukan prediksi pada data testing yang telah distandarisasi menggunakan model svm_lin yang telah dilatih.
- acc_svm_lin = accuracy_score(y_test, y_pred_svm_lin): Menghitung akurasi model dengan membandingkan prediksi dengan nilai sebenarnya.
- print("Test set accuracy: {:.2f}".format(acc_svm_lin)): Menampilkan akurasi model, diformat dengan dua angka di belakang koma.
- print(f"Test set accuracy: {acc_svm_lin}"): Menampilkan akurasi model.

**Training dengan SVM RBF**

In [ ]:
# Model SVM RBF tanpa tunnning hyperparameter
svm_rbf = SVC(kernel='rbf')

# Fit ke model
svm_rbf.fit(X_train_std, y_train)

# Prediksi
y_pred_svm_rbf = svm_rbf.predict(X_test_std)

# Evaluasi akurasi testing data
acc_svm_rbf = accuracy_score(y_test, y_pred_svm_rbf)

# Print hasil evaluasi
print("Test set accuracy: {:.2f}".format(acc_svm_rbf))
print(f"Test set accuracy: {acc_svm_rbf}")

Test set accuracy: 0.72
Test set accuracy: 0.7229437229437229


Kode ini melatih model Support Vector Machine (SVM) dengan kernel Radial Basis Function (RBF) (svm_rbf) menggunakan data yang telah distandarisasi, kemudian mengevaluasi akurasi model pada data testing.

- svm_rbf = SVC(kernel='rbf'): Membuat objek svm_rbf dari kelas SVC dengan kernel RBF.
  - kernel='rbf': Menentukan bahwa model SVM akan menggunakan kernel RBF. Kernel RBF dapat memodelkan hubungan non-linear dalam data.
- svm_rbf.fit(X_train_std, y_train): Melatih model svm_rbf menggunakan data training yang telah distandarisasi.
- y_pred_svm_rbf = svm_rbf.predict(X_test_std): Melakukan prediksi pada data testing yang telah distandarisasi menggunakan model svm_rbf.
- acc_svm_rbf = accuracy_score(y_test, y_pred_svm_rbf): Menghitung akurasi model dengan membandingkan prediksi dengan nilai sebenarnya.
- print("Test set accuracy: {:.2f}".format(acc_svm_rbf)): Menampilkan akurasi model, diformat dengan dua angka di belakang koma.
- print(f"Test set accuracy: {acc_svm_rbf}"): Menampilkan akurasi model.

**Training dengan Voting**

In [ ]:
# Definisikan algoritma yang akan digunakan untuk voting

clf1 = GaussianNB()
clf2 = SVC(kernel='linear')
clf3 = SVC(kernel='rbf', probability=True)

# model hard voting
voting = VotingClassifier(estimators=[('GaussianNB', clf1), ('SVM-LIN', clf2), ('SVM-RBF', clf3)], voting='hard')

# Fit model
voting.fit(X_train_std, y_train)

# Prediksi
y_pred_vt1 = voting.predict(X_test_std)

# Evaluasi akurasi testing data
acc_vt1 = accuracy_score(y_test, y_pred_vt1)

# Print hasil evaluasi
print('Voting Hard')
print("Test set accuracy: {:.2f}".format(acc_vt1))
print(f"Test set accuracy: {acc_vt1}")

Voting Hard
Test set accuracy: 0.74
Test set accuracy: 0.7402597402597403


membuat dan melatih model ensemble Voting Classifier dengan strategi hard voting, menggabungkan tiga model: Gaussian Naive Bayes, SVM dengan kernel linear, dan SVM dengan kernel RBF.

- clf1 = GaussianNB(): Membuat objek clf1 untuk model Gaussian Naive Bayes.
- clf2 = SVC(kernel='linear'): Membuat objek clf2 untuk model SVM dengan kernel linear.
- clf3 = SVC(kernel='rbf', probability=True): Membuat objek clf3 untuk model SVM dengan kernel RBF.
  - probability=True: Parameter ini diperlukan untuk soft voting (meskipun kode ini menggunakan hard voting), agar model SVM dapat menghitung probabilitas kelas.
- voting = VotingClassifier(estimators=[('GaussianNB', clf1), ('SVM-LIN', clf2), ('SVM-RBF', clf3)], voting='hard'): Membuat objek voting dari kelas VotingClassifier.
  - estimators: List yang berisi tuple dari nama estimator dan objek estimator.
  - voting='hard': Menentukan strategi hard voting, di mana kelas prediksi ditentukan berdasarkan suara terbanyak dari estimator.
- voting.fit(X_train_std, y_train): Melatih model voting menggunakan data training yang telah distandarisasi.
- y_pred_vt1 = voting.predict(X_test_std): Melakukan prediksi menggunakan model voting yang telah dilatih.
- acc_vt1 = accuracy_score(y_test, y_pred_vt1): Menghitung akurasi model voting.
- print('Voting Hard'): Menampilkan teks "Voting Hard" untuk memberi label pada output.
- print("Test set accuracy: {:.2f}".format(acc_vt1)): Menampilkan akurasi model hard voting (acc_vt1), diformat dengan dua angka di belakang koma.
- print(f"Test set accuracy: {acc_vt1}"): Menampilkan akurasi model hard voting.

# TUGAS



**Tugas 1**

Terdapat dataset mushroom. Berdasarkan dataset yang tersebut, bandingkan peforma antara algoritma Decision Tree dan RandomForest. Gunakan tunning hyperparameter untuk mendapatkan parameter dan akurasi yang terbaik.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # import DT
from sklearn.ensemble import RandomForestClassifier # import RandomForest
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder


- import numpy as np: Mengimpor library NumPy dan memberikan alias np, digunakan untuk operasi numerik seperti array multidimensi.
- import pandas as pd: Mengimpor library Pandas dan memberikan alias pd, digunakan untuk manipulasi dan analisis data, khususnya struktur data DataFrame.
- from sklearn.tree import DecisionTreeClassifier: Mengimpor kelas DecisionTreeClassifier dari modul sklearn.tree, digunakan untuk membuat model Decision Tree.
- from sklearn.ensemble import RandomForestClassifier: Mengimpor kelas RandomForestClassifier dari modul sklearn.ensemble, digunakan untuk membuat model Random Forest.
- from sklearn.model_selection import train_test_split, GridSearchCV: Mengimpor fungsi train_test_split dan GridSearchCV dari modul sklearn.model_selection.
  - train_test_split digunakan untuk membagi dataset menjadi data training dan data testing.
  - GridSearchCV digunakan untuk mencari kombinasi hyperparameter terbaik untuk model machine learning.
- from sklearn.metrics import accuracy_score, classification_report: Mengimpor fungsi accuracy_score dan classification_report dari modul sklearn.metrics, digunakan untuk mengevaluasi performa model.
- from sklearn.preprocessing import LabelEncoder: Mengimpor kelas LabelEncoder dari modul sklearn.preprocessing, digunakan untuk encoding label, mengubah nilai kategorikal menjadi numerik.

In [ ]:
# Load data
df = pd.read_csv('/content/drive/MyDrive/machine learning/js5/mushrooms.csv')

df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


- df = pd.read_csv() : digunakan untuk memuat data dari file csv ke dalam Pandas DataFrame df
- df.head() : menampilkan 5 baris pertama DataFrame

In [ ]:
# Cek kolom null
df.isnull().sum()

,0
class,0
cap-shape,0
cap-surface,0
cap-color,0
bruises,0
odor,0
gill-attachment,0
gill-spacing,0
gill-size,0
gill-color,0


Menghitung jumlah nilai yang hilang (missing values) di setiap kolom DataFrame df.

- df.isnull(): Mengembalikan DataFrame boolean dengan nilai True untuk setiap sel yang mengandung nilai yang hilang (NaN) dan False untuk sel yang tidak hilang.
- .sum(): Menjumlahkan nilai True di setiap kolom (True dianggap sebagai 1 dan False sebagai 0). Hasilnya adalah jumlah nilai yang hilang di setiap kolom.

In [ ]:
label_encoder = LabelEncoder()
for column in df.columns:
    df[column] = label_encoder.fit_transform(df[column])

Kode ini menerapkan Label Encoding pada semua kolom di DataFrame df. Label Encoding mengubah nilai kategorikal menjadi nilai numerik sehingga data dapat diproses oleh algoritma machine learning.

- label_encoder = LabelEncoder(): Membuat objek label_encoder dari kelas LabelEncoder.
- for column in df.columns:: Looping untuk setiap kolom dalam DataFrame df.
  - df[column] = label_encoder.fit_transform(df[column]): Melakukan Label Encoding pada kolom saat ini.
    - fit_transform: Mempelajari mapping dari nilai kategorikal ke nilai numerik dan langsung menerapkannya pada kolom.

In [ ]:
X = df.drop('class', axis=1)
y = df['class']

Kode ini memisahkan fitur (X) dan target (y) dari DataFrame df untuk mempersiapkan data bagi model machine learning.

- X = df.drop('class', axis=1):
  - Membuat variabel X yang berisi semua kolom kecuali kolom 'class'.
  - drop('class', axis=1): Menghapus kolom 'class' dari DataFrame. axis=1 menunjukkan bahwa operasi dilakukan pada kolom.
  - Variabel X akan digunakan sebagai fitur (input) untuk model.
- y = df['class']:
  - Membuat variabel y yang berisi kolom 'class'.
  - Variabel y akan digunakan sebagai target (output) untuk model.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

- X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1):
  - Membagi data fitur (X) dan target (y) menjadi empat set: data latih fitur (X_train), data uji fitur (X_test), data latih target (y_train), dan data uji target (y_test).
  - test_size=0.2: Menentukan bahwa 20% data akan digunakan sebagai data testing. 80% sisanya akan digunakan sebagai data training.
  - random_state=1: Menetapkan seed untuk generator angka acak. Ini memastikan bahwa pembagian data akan sama setiap kali kode dijalankan, sehingga hasilnya dapat direproduksi.

**Decision Tree**

In [ ]:
param_grid_dt = {
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

dt_classifier = DecisionTreeClassifier(random_state=42)
grid_search_dt = GridSearchCV(estimator=dt_classifier, param_grid=param_grid_dt, cv=5, n_jobs=-1, scoring='accuracy')
grid_search_dt.fit(X_train, y_train)

best_params_dt = grid_search_dt.best_params_
y_pred_dt = grid_search_dt.best_estimator_.predict(X_test)
test_accuracy_dt = accuracy_score(y_test, y_pred_dt)

print(f"Decision Tree Test Accuracy: {test_accuracy_dt:.2f}")

Decision Tree Test Accuracy: 1.00


- param_grid_dt

  - max_depth: Batas kedalaman pohon. Pilihan nilainya adalah [5, 10, 15, 20, None]. Kedalaman yang lebih besar memungkinkan model menangkap pola yang lebih kompleks, tetapi juga dapat menyebabkan overfitting.
  - min_samples_split: Jumlah minimal sampel untuk membagi sebuah node. Dicoba dengan nilai [2, 5, 10].
  - min_samples_leaf: Jumlah minimal sampel yang diperlukan untuk menjadi sebuah daun. Dicoba dengan nilai [1, 2, 4].

- dt_classifier

Sebuah model Decision Tree dibuat dengan random_state=42 untuk memastikan hasil yang dapat direproduksi.

- GridSearchCV digunakan untuk mencoba semua kombinasi parameter yang didefinisikan dalam param_grid_dt.

  - estimator=dt_classifier: Model yang akan di-tune adalah Decision Tree.
  - param_grid=param_grid_dt: Grid hyperparameter yang akan diuji.
  - cv=5: Melakukan cross-validation sebanyak 5 kali untuk setiap kombinasi parameter, membantu menghindari overfitting.
  - n_jobs=-1: Menggunakan semua CPU core untuk komputasi paralel agar lebih cepat.
  - scoring='accuracy': Metode penilaian menggunakan akurasi.

- grid_search_dt.fit(X_train, y_train): Model dilatih pada data latih dan melakukan cross-validation untuk menemukan parameter terbaik.

- best_params_dt = grid_search_dt.best_params_: Mendapatkan kombinasi hyperparameter terbaik yang ditemukan oleh GridSearchCV.
- y_pred_dt = grid_search_dt.best_estimator_.predict(X_test): Menggunakan model terbaik (dengan parameter yang dioptimalkan) untuk memprediksi data uji.
- test_accuracy_dt = accuracy_score(y_test, y_pred_dt): Menghitung akurasi model pada data uji.

- print(f"Decision Tree Test Accuracy: {test_accuracy_dt:.2f}"): Menampilkan akurasi model Decision Tree pada set data uji dalam format desimal dua tempat.

**Random Forest**

In [ ]:
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_classifier = RandomForestClassifier(random_state=42)
grid_search_rf = GridSearchCV(estimator=rf_classifier, param_grid=param_grid_rf, cv=5, n_jobs=-1, scoring='accuracy')
grid_search_rf.fit(X_train, y_train)

# Get best parameters and evaluate on the test set
best_params_rf = grid_search_rf.best_params_
y_pred_rf = grid_search_rf.best_estimator_.predict(X_test)
test_accuracy_rf = accuracy_score(y_test, y_pred_rf)

print(f"RandomForest Test Accuracy: {test_accuracy_rf:.2f}")

RandomForest Test Accuracy: 1.00


- param_grid_rf -->
Grid ini mendefinisikan kombinasi parameter yang akan dicoba dalam proses tuning untuk RandomForest:

  - n_estimators: Jumlah pohon yang akan dibuat oleh Random Forest. Dicoba dengan nilai [50, 100, 200].
  - max_depth: Kedalaman maksimal dari setiap pohon. Dicoba dengan nilai [5, 10, 15, 20, None]. Semakin dalam pohon, semakin kompleks modelnya.
  - min_samples_split: Jumlah minimal sampel yang diperlukan untuk membagi sebuah node. Dicoba dengan nilai [2, 5, 10]. Nilai yang lebih besar membuat pohon lebih simpel dan cenderung menghindari overfitting.
  - min_samples_leaf: Jumlah minimal sampel di setiap daun (node akhir). Dicoba dengan nilai [1, 2, 4].

- rf_classifier -->
Sebuah RandomForestClassifier diinisialisasi dengan random_state=42 untuk memastikan hasil yang dapat diulang.

- GridSearchCV digunakan untuk mencoba semua kombinasi dari hyperparameter yang didefinisikan dalam param_grid_rf.
Argumen:
  - estimator=rf_classifier: Model yang akan dioptimalkan adalah RandomForestClassifier.
  - param_grid=param_grid_rf: Daftar kombinasi hyperparameter yang akan diuji.
  - cv=5: Melakukan cross-validation sebanyak 5 kali untuk setiap kombinasi hyperparameter, yang berguna untuk mencegah overfitting.
  - n_jobs=-1: Menggunakan semua CPU core yang tersedia untuk komputasi paralel agar lebih cepat.
  - scoring='accuracy': Menggunakan akurasi sebagai metrik penilaian.

grid_search_rf.fit(X_train, y_train): Melatih model dengan data latih menggunakan berbagai kombinasi parameter yang didefinisikan dalam param_grid_rf dan menerapkan cross-validation.

- best_params_rf = grid_search_rf.best_params_: Mendapatkan kombinasi hyperparameter terbaik berdasarkan hasil cross-validation.
- y_pred_rf = grid_search_rf.best_estimator_.predict(X_test): Menggunakan model Random Forest terbaik untuk memprediksi data uji.
- test_accuracy_rf = accuracy_score(y_test, y_pred_rf): Menghitung akurasi dari model Random Forest pada data uji.

- print(f"RandomForest Test Accuracy: {test_accuracy_rf:.2f}"): Menampilkan akurasi model Random Forest yang sudah dioptimalkan pada set data uji, dibulatkan hingga dua tempat desimal.


**Tugas 2**

Terdapat dataset mushroom. Berdasarkan dataset tersebut, bandingkan peforma antara algoritma Decision Tree dan AdaBoost. Gunakan tunning hyperparameter untuk mendapatkan parameter dan akurasi yang terbaik.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

- import pandas as pd: Mengimpor library Pandas dan memberikan alias pd. Pandas digunakan untuk manipulasi dan analisis data, khususnya untuk struktur data DataFrame.

- from sklearn.model_selection import train_test_split, GridSearchCV: Mengimpor train_test_split dan GridSearchCV dari modul sklearn.model_selection.

- train_test_split: Fungsi yang digunakan untuk membagi dataset menjadi subset training dan testing.

- GridSearchCV: Teknik yang digunakan untuk menemukan kombinasi hyperparameter terbaik untuk model machine learning.

- from sklearn.tree import DecisionTreeClassifier: Mengimpor kelas DecisionTreeClassifier dari modul sklearn.tree. DecisionTreeClassifier digunakan untuk membuat model Decision Tree, yang sering digunakan sebagai base estimator untuk AdaBoost.

- from sklearn.ensemble import AdaBoostClassifier: Mengimpor kelas AdaBoostClassifier dari modul sklearn.ensemble. AdaBoostClassifier digunakan untuk membuat model AdaBoost. AdaBoost adalah metode ensemble learning yang menggabungkan beberapa weak learner (seperti Decision Tree) untuk membuat strong learner.

- from sklearn.metrics import accuracy_score: Mengimpor fungsi accuracy_score dari modul sklearn.metrics. accuracy_score digunakan untuk menghitung akurasi model klasifikasi.

- from sklearn.preprocessing import LabelEncoder: Mengimpor kelas LabelEncoder dari modul sklearn.preprocessing. LabelEncoder digunakan untuk mengonversi label kategorikal menjadi label numerik, yang dibutuhkan oleh beberapa algoritma machine learning.

In [ ]:
# Load data
df = pd.read_csv('/content/drive/MyDrive/machine learning/js5/mushrooms.csv')

df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


- df = pd.read_csv() : digunakan untuk memuat data dari file csv ke dalam Pandas DataFrame df
- df.head() : menampilkan 5 baris pertama DataFrame

In [ ]:
# Cek kolom null
df.isnull().sum()

,0
class,0
cap-shape,0
cap-surface,0
cap-color,0
bruises,0
odor,0
gill-attachment,0
gill-spacing,0
gill-size,0
gill-color,0


Menghitung jumlah nilai yang hilang (missing values) di setiap kolom DataFrame df.

- df.isnull(): Mengembalikan DataFrame boolean dengan nilai True untuk setiap sel yang mengandung nilai yang hilang (NaN) dan False untuk sel yang tidak hilang.
- .sum(): Menjumlahkan nilai True di setiap kolom (True dianggap sebagai 1 dan False sebagai 0). Hasilnya adalah jumlah nilai yang hilang di setiap kolom.

In [ ]:
label_encoder = LabelEncoder()
for column in df.columns:
    df[column] = label_encoder.fit_transform(df[column])

Kode ini menerapkan Label Encoding pada semua kolom di DataFrame df. Label Encoding mengubah nilai kategorikal menjadi nilai numerik sehingga data dapat diproses oleh algoritma machine learning.

- label_encoder = LabelEncoder(): Membuat objek label_encoder dari kelas LabelEncoder.
- for column in df.columns:: Looping untuk setiap kolom dalam DataFrame df.
  - df[column] = label_encoder.fit_transform(df[column]): Melakukan Label Encoding pada kolom saat ini.
    - fit_transform: Mempelajari mapping dari nilai kategorikal ke nilai numerik dan langsung menerapkannya pada kolom.

In [ ]:
X = df.drop('class', axis=1)
y = df['class']

Kode ini memisahkan fitur (X) dan target (y) dari DataFrame df untuk mempersiapkan data bagi model machine learning.

- X = df.drop('class', axis=1):
Membuat variabel X yang berisi semua kolom kecuali kolom 'class'.
  - drop('class', axis=1): Menghapus kolom 'class' dari DataFrame. axis=1 menunjukkan bahwa operasi dilakukan pada kolom.
  - Variabel X akan digunakan sebagai fitur (input) untuk model.
- y = df['class']:
Membuat variabel y yang berisi kolom 'class'.
  - Variabel y akan digunakan sebagai target (output) untuk model.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

- X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1):
  - Membagi data fitur (X) dan target (y) menjadi empat set: data latih fitur (X_train), data uji fitur (X_test), data latih target (y_train), dan data uji target (y_test).
  - test_size=0.2: Menentukan bahwa 20% data akan digunakan sebagai data testing. 80% sisanya akan digunakan sebagai data training.
  - random_state=1: Menetapkan seed untuk generator angka acak. Ini memastikan bahwa pembagian data akan sama setiap kali kode dijalankan, sehingga hasilnya dapat direproduksi.

**Decision Tree**

In [ ]:
param_grid_dt = {
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

dt_classifier = DecisionTreeClassifier(random_state=42)
grid_search_dt = GridSearchCV(estimator=dt_classifier, param_grid=param_grid_dt, cv=5, n_jobs=-1, scoring='accuracy')
grid_search_dt.fit(X_train, y_train)

best_params_dt = grid_search_dt.best_params_
y_pred_dt = grid_search_dt.best_estimator_.predict(X_test)
test_accuracy_dt = accuracy_score(y_test, y_pred_dt)

print(f"Decision Tree Test Accuracy: {test_accuracy_dt:.2f}")


Decision Tree Test Accuracy: 1.00


- param_grid_dt
  - max_depth: Batas kedalaman maksimum pohon. Pilihan nilainya adalah [5, 10, 15, 20, None], di mana None artinya tidak ada batasan kedalaman.
  - min_samples_split: Jumlah minimal sampel yang diperlukan untuk membagi node internal. Nilai yang dicoba adalah [2, 5, 10].
  - min_samples_leaf: Jumlah minimal sampel yang dibutuhkan pada setiap daun (node akhir). Nilai yang dicoba adalah [1, 2, 4].
  - Grid ini mendefinisikan kombinasi hyperparameter yang akan diuji oleh GridSearchCV untuk menentukan mana yang menghasilkan performa terbaik.

- dt_classifier = DecisionTreeClassifier(random_state=42)
  - Membuat model DecisionTreeClassifier dengan parameter random_state=42, yang digunakan untuk memastikan hasil yang konsisten dan dapat direproduksi setiap kali model dijalankan.

- grid_search_dt = GridSearchCV(estimator=dt_classifier, param_grid=param_grid_dt, cv=5, n_jobs=-1, scoring='accuracy')
  - estimator=dt_classifier: Model yang akan dioptimalkan adalah Decision Tree.
  - param_grid=param_grid_dt: Grid dari hyperparameter yang akan diuji.
  - cv=5: Melakukan cross-validation dengan 5 lipatan (folds), yang berarti data akan dibagi menjadi 5 bagian untuk evaluasi.
  - n_jobs=-1: Menggunakan semua core CPU yang tersedia untuk komputasi paralel agar proses lebih cepat.
  - scoring='accuracy': Menentukan bahwa akurasi digunakan sebagai metrik untuk menilai performa model.

- grid_search_dt.fit(X_train, y_train)
  - Model dilatih menggunakan data latih (X_train, y_train) dengan mencoba semua kombinasi hyperparameter yang ditentukan dalam param_grid_dt melalui proses cross-validation.

- best_params_dt = grid_search_dt.best_params_
y_pred_dt = grid_search_dt.best_estimator_.predict(X_test)
  - best_params_dt = grid_search_dt.best_params_: Mendapatkan kombinasi hyperparameter terbaik yang ditemukan oleh GridSearchCV berdasarkan hasil cross-validation.
  - y_pred_dt = grid_search_dt.best_estimator_.predict(X_test): Menggunakan model dengan parameter terbaik untuk memprediksi hasil pada data uji (X_test).

- test_accuracy_dt = accuracy_score(y_test, y_pred_dt)
print(f"Decision Tree Test Accuracy: {test_accuracy_dt:.2f}")
  - accuracy_score(y_test, y_pred_dt): Menghitung akurasi prediksi model pada data uji (X_test) dibandingkan dengan label sebenarnya (y_test).
  - print: Menampilkan akurasi model Decision Tree pada data uji dalam format dua desimal.

**AdaBoost**

In [ ]:
param_grid_ab = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 1]
}

ada_classifier = AdaBoostClassifier(estimator=DecisionTreeClassifier(), random_state=42) # Changed 'base_estimator' to 'estimator'
grid_search_ab = GridSearchCV(estimator=ada_classifier, param_grid=param_grid_ab, cv=5, n_jobs=-1, scoring='accuracy')
grid_search_ab.fit(X_train, y_train)

best_params_ab = grid_search_ab.best_params_
y_pred_ab = grid_search_ab.best_estimator_.predict(X_test)
test_accuracy_ab = accuracy_score(y_test, y_pred_ab)

print(f"AdaBoost Test Accuracy: {test_accuracy_ab:.2f}")

AdaBoost Test Accuracy: 1.00


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


- param_grid_ab
  - n_estimators: Jumlah estimators (weak learners, yang dalam hal ini adalah Decision Trees) yang digunakan oleh AdaBoost. Nilai yang dicoba adalah [50, 100, 200].
  - learning_rate: Tingkat pembelajaran yang mengatur kontribusi setiap weak learner. Nilai yang diuji adalah [0.01, 0.1, 1]. Nilai kecil akan membuat model lebih konservatif, sementara nilai yang besar akan membuat model belajar lebih cepat.

- ada_classifier = AdaBoostClassifier(estimator=DecisionTreeClassifier(), random_state=42)
  - AdaBoostClassifier: Algoritma ensemble boosting yang menggunakan DecisionTreeClassifier sebagai estimator dasarnya.
  - random_state=42: Menetapkan seed untuk memastikan hasil yang dapat direproduksi.

- grid_search_ab = GridSearchCV(estimator=ada_classifier, param_grid=param_grid_ab, cv=5, n_jobs=-1, scoring='accuracy')
  - estimator=ada_classifier: Model yang dioptimalkan adalah AdaBoost dengan Decision Tree sebagai estimasi dasarnya.
  - param_grid=param_grid_ab: Grid dari hyperparameter yang akan diuji untuk menemukan kombinasi terbaik.
  - cv=5: Melakukan cross-validation dengan 5 lipatan untuk mendapatkan performa yang lebih robust.
  - n_jobs=-1: Menggunakan semua core CPU yang tersedia untuk mempercepat proses tuning.
  - scoring='accuracy': Metrik yang digunakan untuk menilai performa model adalah akurasi.

- grid_search_ab.fit(X_train, y_train)
  - Melatih model menggunakan data latih (X_train, y_train) dan mencoba semua kombinasi hyperparameter yang ditentukan dalam param_grid_ab.

- best_params_ab = grid_search_ab.best_params_
y_pred_ab = grid_search_ab.best_estimator_.predict(X_test)
  - best_params_ab = grid_search_ab.best_params_: Mendapatkan kombinasi hyperparameter terbaik yang ditemukan oleh GridSearchCV berdasarkan hasil cross-validation.
  - y_pred_ab = grid_search_ab.best_estimator_.predict(X_test): Menggunakan model dengan parameter terbaik untuk memprediksi hasil pada data uji (X_test).

- test_accuracy_ab = accuracy_score(y_test, y_pred_ab)
print(f"AdaBoost Test Accuracy: {test_accuracy_ab:.2f}")
  - accuracy_score(y_test, y_pred_ab): Menghitung akurasi prediksi pada data uji (X_test) dibandingkan dengan label sebenarnya (y_test).
  - print: Menampilkan hasil akurasi model AdaBoost pada data uji dalam dua desimal.

**Tugas 3**

Dengan menggunakan dataset diabetes, buatlah ensemble voting dengan algoritma

1. Logistic Regression

2. SVM kernel polynomial

3. Decission Tree

Anda boleh melakukan eksplorasi dengan melakukan tunning hyperparameter

In [ ]:
import pandas as pd

# Load the dataset
file_path = '/content/drive/MyDrive/machine learning/js5/diabetes.csv'
data = pd.read_csv(file_path)

data.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


- dbt = pd.read_csv() : digunakan untuk memuat data dari file csv ke dalam Pandas DataFrame dbt
- dbt.head() : menampilkan 5 baris pertama DataFrame

In [ ]:
# Cek kolom null
data.isnull().sum()

,0
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0
Outcome,0


- dbt.isnull(): Mengembalikan DataFrame boolean dengan nilai True untuk setiap sel yang mengandung nilai yang hilang (NaN) dan False untuk sel yang tidak hilang.
- .sum(): Menjumlahkan nilai True di setiap kolom (True dianggap sebagai 1 dan False sebagai 0). Hasilnya adalah jumlah nilai yang hilang di setiap kolom.

In [ ]:
# Pada kasus ini, agak tidak masuk akal jika beberapa parameter bernilai 0
# sebagai contoh adalah nilai 'Glucose', 'BloodPlessure' ataupun 'Insulin'.
# Sekecil apapun nilainya, setiap manusia yang hidup pasti miliki nilai-nilai tersebut

# Kita akan manipulasi nilai yang 0 dengan melakukan 'imputasi' atau mengganti nilainya dengan nilai sintetis
# Pada kasus ini, kita akan menggunakan nilai mean

# Cek kolom neng nilai 0
feature_columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
for column in feature_columns:
    print("============================================")
    print(f"{column} ==> Missing zeros : {len(data.loc[data[column] == 0])}")

Pregnancies ==> Missing zeros : 111
Glucose ==> Missing zeros : 5
BloodPressure ==> Missing zeros : 35
SkinThickness ==> Missing zeros : 227
Insulin ==> Missing zeros : 374
BMI ==> Missing zeros : 11
DiabetesPedigreeFunction ==> Missing zeros : 0
Age ==> Missing zeros : 0


- feature_columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']: Mendefinisikan list feature_columns yang berisi nama-nama kolom fitur yang ingin diperiksa.
- for column in feature_columns:: Looping untuk setiap kolom dalam feature_columns.
  - print("============================================"): Menampilkan garis pemisah untuk memperjelas output.
  - print(f"{column} ==> Missing zeros : {len(dbt.loc[dbt[column] == 0])}"): Menampilkan nama kolom dan jumlah nilai nol yang hilang pada kolom tersebut.
    - dbt.loc[dbt[column] == 0]: Memilih baris di dbt di mana nilai kolom column sama dengan 0.
    - len(...): Menghitung jumlah baris yang terpilih.

Kode ini membantu dalam identifikasi data yang hilang yang direpresentasikan sebagai nilai nol. Nilai nol pada beberapa kolom, seperti 'Glucose' atau 'BloodPressure', mungkin mengindikasikan data yang hilang dan perlu ditangani sebelum digunakan untuk training model machine learning.

In [ ]:
# Impute nilai 0 dengan mean
from sklearn.impute import SimpleImputer

fill_values = SimpleImputer(missing_values=0, strategy="mean", copy=False)

data[feature_columns] = fill_values.fit_transform(data[feature_columns])

Kode ini menangani nilai yang hilang (missing values) yang direpresentasikan sebagai 0 pada kolom-kolom fitur dalam DataFrame dbt menggunakan SimpleImputer dari scikit-learn.

- from sklearn.impute import SimpleImputer: Mengimpor kelas SimpleImputer dari modul sklearn.impute. SimpleImputer digunakan untuk mengisi nilai yang hilang dengan strategi sederhana seperti mean, median, atau modus.

- fill_values = SimpleImputer(missing_values=0, strategy="mean", copy=False): Membuat objek fill_values dari kelas SimpleImputer dengan parameter:

  - missing_values=0: Menentukan bahwa nilai 0 dianggap sebagai nilai yang hilang.
  - strategy="mean": Menentukan bahwa nilai yang hilang akan diisi dengan nilai rata-rata (mean) dari kolom tersebut.
  - copy=False: Menentukan bahwa imputasi akan dilakukan secara inplace pada DataFrame asli.

- dbt[feature_columns] = fill_values.fit_transform(dbt[feature_columns]): Mengisi nilai yang hilang pada kolom-kolom yang ditentukan di feature_columns menggunakan objek fill_values.

- fit_transform: Metode ini akan menghitung nilai rata-rata dari setiap kolom (fit) dan kemudian mengganti nilai yang hilang dengan nilai rata-rata tersebut (transform).

In [ ]:
X = data[feature_columns]
y = data.Outcome

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

memisahkan fitur dan target dari DataFrame dbt, kemudian membagi data menjadi set training dan testing:

- X = dbt[feature_columns]: Membuat variabel X yang berisi fitur-fitur (input) untuk model machine learning. Fitur-fitur ini diambil dari kolom-kolom yang ada di feature_columns.
- y = dbt.Outcome: Membuat variabel y yang berisi target (output) untuk model. Target diambil dari kolom 'Outcome'.
- X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42): Membagi data (X dan y) menjadi set training dan testing menggunakan fungsi train_test_split.
  - test_size=0.3: Menentukan bahwa 30% data akan digunakan untuk testing.
  - random_state=42: Menjamin pembagian data yang sama setiap kali kode dijalankan, sehingga hasilnya dapat direproduksi.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Define hyperparameters for Logistic Regression
param_grid_lr = {
    'C': [0.01, 0.1, 1, 10],
    'penalty': ['l2']
}

# Define hyperparameters for SVM with polynomial kernel
param_grid_svm = {
    'C': [0.01, 0.1, 1, 10],
    'degree': [2, 3, 4],  # Polynomial degrees
    'kernel': ['poly']
}

# Define hyperparameters for Decision Tree
param_grid_dt = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10]
}




1. Hyperparameter untuk Logistic Regression

  - C: Ini adalah parameter regularisasi yang mengontrol kekuatan regularisasi pada Logistic Regression. Semakin besar nilai C, semakin lemah regularisasinya, yang berarti model bisa menjadi lebih kompleks. Nilai yang dicoba adalah [0.01, 0.1, 1, 10].

  - penalty: Penalti yang digunakan untuk regularisasi. l2 mengacu pada penalti Ridge atau norma Euclidean. Hanya penalti l2 yang dicoba di sini.

2. Hyperparameter untuk SVM dengan Kernel Polinomial

  - C: Sama seperti di Logistic Regression, C mengontrol regularisasi untuk SVM. Nilai C yang lebih besar memungkinkan model menjadi lebih kompleks. Nilai yang dicoba adalah [0.01, 0.1, 1, 10].

  - degree: Ini adalah derajat polinomial untuk kernel polinomial pada SVM. SVM dengan kernel polinomial akan membuat decision boundary berdasarkan fungsi polinomial. Nilai yang dicoba adalah derajat polinomial [2, 3, 4].

  - kernel: Jenis kernel yang digunakan oleh SVM. Dalam kasus ini, kernel yang digunakan adalah poly (polinomial).

3. Hyperparameter untuk Decision Tree

  - max_depth: Ini mengontrol kedalaman maksimal dari decision tree. Decision tree yang lebih dalam cenderung memodelkan data lebih baik (tapi juga berisiko overfitting). Nilai yang dicoba adalah [3, 5, 7, 10].

  - min_samples_split: Ini adalah jumlah minimum sampel yang diperlukan untuk memisahkan node internal. Jika nilai ini lebih tinggi, pohon akan cenderung lebih sederhana (menghindari overfitting). Nilai yang dicoba adalah [2, 5, 10].


In [ ]:
# Initialize the classifiers
lr = LogisticRegression(random_state=42)
svm = SVC(random_state=42, probability=True)
dt = DecisionTreeClassifier(random_state=42)

# Tuning Logistic Regression
grid_search_lr = GridSearchCV(estimator=lr, param_grid=param_grid_lr, cv=5, n_jobs=-1, scoring='accuracy')
grid_search_lr.fit(X_train, y_train)
best_lr = grid_search_lr.best_estimator_

# Tuning SVM with polynomial kernel
grid_search_svm = GridSearchCV(estimator=svm, param_grid=param_grid_svm, cv=5, n_jobs=-1, scoring='accuracy')
grid_search_svm.fit(X_train, y_train)
best_svm = grid_search_svm.best_estimator_

# Tuning Decision Tree
grid_search_dt = GridSearchCV(estimator=dt, param_grid=param_grid_dt, cv=5, n_jobs=-1, scoring='accuracy')
grid_search_dt.fit(X_train, y_train)
best_dt = grid_search_dt.best_estimator_

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


1. Inisialisasi Model Klasifikasi

  - lr (Logistic Regression): Menginisialisasi model Logistic Regression dengan parameter random_state=42 untuk mendapatkan hasil yang konsisten.

  - svm (Support Vector Classifier): Menginisialisasi SVM dengan kernel='poly' (sesuai hyperparameter tuning). Parameter probability=True memungkinkan kita mendapatkan probabilitas kelas saat prediksi.

  - dt (Decision Tree Classifier): Menginisialisasi model Decision Tree dengan random_state=42 agar hasil tetap konsisten di setiap eksekusi.

2. Hyperparameter Tuning untuk Logistic Regression

  - GridSearchCV: GridSearchCV digunakan untuk menguji berbagai kombinasi hyperparameter yang didefinisikan di param_grid_lr untuk Logistic Regression.

    - param_grid_lr adalah grid dari hyperparameter yang akan diuji.
    - cv=5 berarti menggunakan 5-fold cross-validation.
    - scoring='accuracy': Metrik evaluasi yang digunakan adalah akurasi.
  - fit(X_train, y_train): Menerapkan GridSearchCV pada data pelatihan. Proses ini akan mencari hyperparameter terbaik dari grid yang disediakan.

  - best_lr: Menyimpan estimator Logistic Regression dengan kombinasi hyperparameter terbaik yang ditemukan oleh GridSearchCV.

3. Hyperparameter Tuning untuk SVM dengan Kernel Polinomial

  Proses yang sama dilakukan pada SVM:

  - Menggunakan param_grid_svm yang berisi hyperparameter seperti C, degree, dan kernel.
  - fit(X_train, y_train): Menerapkan GridSearchCV pada data pelatihan untuk mencari kombinasi hyperparameter terbaik untuk SVM dengan kernel polinomial.

  - best_svm: Menyimpan estimator SVM terbaik yang ditemukan oleh GridSearchCV.

4. Hyperparameter Tuning untuk Decision Tree

  - param_grid_dt mendefinisikan hyperparameter seperti max_depth dan min_samples_split untuk Decision Tree.

  - fit(X_train, y_train): Menerapkan GridSearchCV pada Decision Tree dengan data pelatihan, mencari hyperparameter terbaik.

  - best_dt: Menyimpan model Decision Tree terbaik setelah pencarian hyperparameter.


In [ ]:
# Logistic Regression Prediction and Evaluation
y_pred_lr = best_lr.predict(X_test)
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print(f"Logistic Regression Test Accuracy: {accuracy_lr:.2f}")

# SVM Prediction and Evaluation
y_pred_svm = best_svm.predict(X_test)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f"SVM (Polynomial Kernel) Test Accuracy: {accuracy_svm:.2f}")

# Decision Tree Prediction and Evaluation
y_pred_dt = best_dt.predict(X_test)
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print(f"Decision Tree Test Accuracy: {accuracy_dt:.2f}")

Logistic Regression Test Accuracy: 0.74
SVM (Polynomial Kernel) Test Accuracy: 0.75
Decision Tree Test Accuracy: 0.74


In [ ]:
from sklearn.ensemble import VotingClassifier

# Create Voting Classifier
voting_clf = VotingClassifier(
    estimators=[('lr', best_lr), ('svm', best_svm), ('dt', best_dt)],
    voting='soft'  # Soft voting for probability-based predictions
)

# Train the voting classifier
voting_clf.fit(X_train, y_train)

# Predict and evaluate
y_pred = voting_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
print(f"Ensemble Voting Classifier Accuracy: {accuracy:.2f}")

Ensemble Voting Classifier Accuracy: 0.75


1. Membuat Voting Classifier

  - VotingClassifier adalah metode ensemble yang menggabungkan prediksi dari beberapa model untuk membuat prediksi akhir.
    - estimators=[('lr', best_lr), ('svm', best_svm), ('dt', best_dt)]: Ini adalah daftar pasangan nama-model dari tiga model yang sudah di-tuning sebelumnya.
      - best_lr: Model Logistic Regression terbaik.
      - best_svm: Model SVM terbaik dengan kernel polinomial.
      - best_dt: Model Decision Tree terbaik.
  - voting='soft': Menggunakan soft voting, yang berarti model akan menggabungkan probabilitas prediksi dari setiap model untuk membuat prediksi akhir. Soft voting biasanya menghasilkan kinerja yang lebih baik dibandingkan hard voting (yang menggunakan prediksi kelas mayoritas).

2. Melatih Voting Classifier

  - voting_clf.fit(X_train, y_train): Melatih model Voting Classifier pada data pelatihan (X_train dan y_train). Masing-masing model di dalam ensemble (Logistic Regression, SVM, dan Decision Tree) dilatih pada dataset ini.

3. Prediksi dan Evaluasi Akurasi

  - y_pred = voting_clf.predict(X_test): Menggunakan model Voting Classifier yang sudah dilatih untuk memprediksi label pada data uji (X_test).

  - accuracy = accuracy_score(y_test, y_pred): Menghitung akurasi dari prediksi pada data uji dengan membandingkan y_pred (label yang diprediksi) dan y_test (label asli dari data uji).